In [10]:
import numpy as np
import pandas as pd

In [11]:
credits = pd.read_csv('credits.csv')

In [12]:
credits

,director_name,release_date,actor_1_name,actor_2_name,actor_3_name,actor_4_name,actor_5_name,actor_6_name


In [13]:
meta = pd.read_csv('movie_metadata.csv')

In [14]:
meta['release_date'] = pd.to_datetime(meta['release_date'], errors='coerce')

KeyError: 'release_date'

In [15]:

meta['year'] = meta['release_date'].dt.year

KeyError: 'release_date'

In [ ]:
meta['year'].value_counts().sort_index()

In [ ]:
# Getting only 2017 movies as we already have movies up to the year 2016 in preprocessing 1 file. 
# We don't have enough data for the movies from 2018, 2019 and 2020. 
# We'll deal with it in the upcoming preprocessing files
new_meta = meta.loc[meta.year == 2017,['genres','id','title','year']]

In [ ]:

new_meta

In [ ]:
new_meta['id'] = new_meta['id'].astype(int)

In [ ]:
data = pd.merge(new_meta, credits, on='id')

In [ ]:
pd.set_option('display.max_colwidth', 75)
data

In [ ]:
# evaluates an expression node or a string containing a Python literal or container display
import ast
data['genres'] = data['genres'].map(lambda x: ast.literal_eval(x))
data['cast'] = data['cast'].map(lambda x: ast.literal_eval(x))
data['crew'] = data['crew'].map(lambda x: ast.literal_eval(x))

In [ ]:
def make_genresList(x):
    gen = []
    st = " "
    for i in x:
        if i.get('name') == 'Science Fiction':
            scifi = 'Sci-Fi'
            gen.append(scifi)
        else:
            gen.append(i.get('name'))
    if gen == []:
        return np.NaN
    else:
        return (st.join(gen))

In [ ]:
data['genres_list'] = data['genres'].map(lambda x: make_genresList(x))

In [ ]:
data['genres_list']

In [ ]:
def get_actor1(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == []:
        return np.NaN
    else:
        return (casts[0])

In [ ]:
data['actor_1_name'] = data['cast'].map(lambda x: get_actor1(x))

In [ ]:
def get_actor2(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=1:
        return np.NaN
    else:
        return (casts[1])

In [ ]:
data['actor_2_name'] = data['cast'].map(lambda x: get_actor2(x))

In [ ]:
data['actor_2_name']

In [ ]:
def get_actor3(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=2:
        return np.NaN
    else:
        return (casts[2])

In [ ]:
data['actor_3_name'] = data['cast'].map(lambda x: get_actor3(x))

In [ ]:
data['actor_3_name']

In [ ]:
def get_directors(x):
    dt = []
    st = " "
    for i in x:
        if i.get('job') == 'Director':
            dt.append(i.get('name'))
    if dt == []:
        return np.NaN
    else:
        return (st.join(dt))

In [ ]:
data['director_name'] = data['crew'].map(lambda x: get_directors(x))

In [ ]:
data['director_name']

In [ ]:
movie = data.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres_list','title']]

In [ ]:
movie

In [ ]:
movie.isna().sum()

In [ ]:
movie = movie.dropna(how='any')

In [ ]:
movie.isna().sum()

In [ ]:
movie = movie.rename(columns={'genres_list':'genres'})
movie = movie.rename(columns={'title':'movie_title'})

In [ ]:
movie['movie_title'] = movie['movie_title'].str.lower()

In [ ]:
movie['comb'] = movie['actor_1_name'] + ' ' + movie['actor_2_name'] + ' '+ movie['actor_3_name'] + ' '+ movie['director_name'] +' ' + movie['genres']

In [ ]:
movie

In [ ]:
old = pd.read_csv('data.csv')

In [ ]:
old

In [ ]:
old['comb'] = old['actor_1_name'] + ' ' + old['actor_2_name'] + ' '+ old['actor_3_name'] + ' '+ old['director_name'] +' ' + old['genres']

In [ ]:
old

In [ ]:
new = old.append(movie)

In [ ]:
new

In [ ]:
new.drop_duplicates(subset ="movie_title", keep = 'last', inplace = True)

In [ ]:
new

In [ ]:
new.to_csv('new_data.csv',index=False)